In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pydicom

from skimage.measure import label,regionprops
from skimage.segmentation import clear_border
import matplotlib.pyplot as plt

## Raw image

In [ ]:
# https://www.kaggle.com/currypurin/osic-image-shape-eda-and-preprocess
def crop_image(img: np.ndarray):
    edge_pixel_value = img[0, 0]
    mask = img != edge_pixel_value
    return img[np.ix_(mask.any(1),mask.any(0))]

In [ ]:
#d = pydicom.dcmread('../input/osic-pulmonary-fibrosis-progression/train/ID00030637202181211009029/100.dcm')
d = pydicom.dcmread('../input/osic-pulmonary-fibrosis-progression/test/ID00419637202311204720264/21.dcm')

In [ ]:
img = crop_image(d.pixel_array)

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow(img)

## Rescale and create binary mask
The bright region inside the lungs are the blood vessels or air. A threshold of -600--850 HU is used at all places because it was found in experiments. 

In [ ]:
right_mask = cv2.imread('../input/osic-generalized-lung-mask/mask/right_mask_simetric.jpg', 0)
left_mask = cv2.imread('../input/osic-generalized-lung-mask/mask/left_mask_simetric.jpg', 0)

In [ ]:
img = (crop_image(d.pixel_array) + d.RescaleIntercept) / d.RescaleSlope
dim = min(img.shape)
cancer_mask = (img > 10) & (img < 400)
lung_mask = (img > -850) & (img < -600)
lung_mask = cv2.bilateralFilter(lung_mask.astype('float32'),int(dim*0.05),int(dim*0.2),int(dim*0.2)) > 0.1

In [ ]:
cancer_mask[~lung_mask] = 0

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow(cancer_mask)
plt.imshow(lung_mask, alpha=0.3)

Based on the previous mask I created a generic lung mask for right and left lung.

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow(right_mask, alpha=0.75)
plt.imshow(left_mask, alpha=0.75);

In [ ]:
right_mask = cv2.resize(right_mask, lung_mask.shape[::-1]).astype('uint8')
left_mask = cv2.resize(left_mask, lung_mask.shape[::-1]).astype('uint8')

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow(cancer_mask)
plt.imshow(lung_mask, alpha=0.4)
plt.imshow(cv2.resize(right_mask, lung_mask.shape[::-1]), alpha=0.25)
plt.imshow(cv2.resize(left_mask, lung_mask.shape[::-1]), alpha=0.25)

## Cleaning border

In [ ]:
cancer_mask = clear_border(cancer_mask)
lung_mask = clear_border(lung_mask)

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow(cancer_mask)
plt.imshow(lung_mask, alpha=0.4)

## Remove small and select nearest to generic lung mask region

In [ ]:
from sklearn.metrics import jaccard_score

In [ ]:
lung_mask_labeled = label(lung_mask)

fig = plt.figure(figsize=(12, 12))

plt.imshow(lung_mask_labeled)

In [ ]:
for i, r in enumerate(regionprops(lung_mask_labeled)):
    _lung_mask = lung_mask.copy()
    m = np.zeros_like(_lung_mask)
    m[r.slice] = 1
    _lung_mask = _lung_mask * m > 0
    riou = jaccard_score(_lung_mask, right_mask > 0, average='micro')
    liou = jaccard_score(_lung_mask, left_mask > 0, average='micro')
    print(f"Region {i}")
    print("\tRight: ", riou, "\n\tLeft: ", liou)
    if liou < 0.1 and riou < 0.1:
        for coordinates in r.coords:                
            lung_mask_labeled[coordinates[0], coordinates[1]] = 0

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow(cancer_mask)
plt.imshow(lung_mask_labeled, alpha=0.5);

In [ ]:
fig = plt.figure(figsize=(12, 12))

plt.imshow((crop_image(d.pixel_array) + d.RescaleIntercept) / d.RescaleSlope)
plt.imshow(lung_mask_labeled, alpha=0.5);

## Conclusion 
The determining of a mask for the lungs is the starting point in the algorithm for determining the volume of the lungs by CT images. The next step is the correct integration of all CT images to determine the volume of the lungs and volume posible cancer regions. 

I trying current approach for many patients and taking a good lungs of mask in a result.

## Reference
* [OSIC / image shape EDA and preprocess](https://www.kaggle.com/currypurin/osic-image-shape-eda-and-preprocess)
* [Unsupervise Lung Detection](https://www.kaggle.com/miklgr500/unsupervise-lung-detection)